# [自定义聚集函数](http://www.postgres.cn/docs/11/xaggr.html#XAGGR-MOVING-AGGREGATES)

所谓聚集函数是指用于从一个输入值的集合计算出一个单一值,它的本质是一个可以被不断迭代的状态计算函数,其形式有点类似函数式编程中的fold,或者大家更熟悉的reduce的概念. 比如`sum`,`avg`就是典型的聚集函数.pg支持的聚集函数可以查看[这张表](http://www.postgres.cn/docs/11/functions-aggregate.html).

在pg中聚集函数一般用在如`GROUP BY`和窗口查询中.

自定义聚集函数中也可以根据定义和使用时的语法不同细分为:

+ 普通聚集函数,其定义方式为

    ```sql
    CREATE AGGREGATE name ( [ argmode ] [ argname ] arg_data_type [ , ... ] ) (
        SFUNC = sfunc,
        STYPE = state_data_type
        [ , SSPACE = state_data_size ]
        [ , FINALFUNC = ffunc ]
        [ , FINALFUNC_EXTRA ]
        [ , FINALFUNC_MODIFY = { READ_ONLY | SHAREABLE | READ_WRITE } ]
        [ , COMBINEFUNC = combinefunc ]
        [ , SERIALFUNC = serialfunc ]
        [ , DESERIALFUNC = deserialfunc ]
        [ , INITCOND = initial_condition ]
        [ , MSFUNC = msfunc ]
        [ , MINVFUNC = minvfunc ]
        [ , MSTYPE = mstate_data_type ]
        [ , MSSPACE = mstate_data_size ]
        [ , MFINALFUNC = mffunc ]
        [ , MFINALFUNC_EXTRA ]
        [ , MINITCOND = minitial_condition ]
        [ , SORTOP = sort_operator ]
        [ , PARALLEL = { SAFE | RESTRICTED | UNSAFE } ]
    )
    ```
    普通聚集可以通过增加特定函数实现两种特殊模式:
    + 移动聚集模式
    + 部分聚集模式

    
+ 有序集聚集,其定义方式为

    ```sql
    CREATE AGGREGATE name ( [ [ argmode ] [ argname ] arg_data_type [ , ... ] ]
                        ORDER BY [ argmode ] [ argname ] arg_data_type [ , ... ] ) (
        SFUNC = sfunc,
        STYPE = state_data_type
        [ , SSPACE = state_data_size ]
        [ , FINALFUNC = ffunc ]
        [ , FINALFUNC_EXTRA ]
        [ , FINALFUNC_MODIFY = { READ_ONLY | SHAREABLE | READ_WRITE } ]
        [ , INITCOND = initial_condition ]
        [ , HYPOTHETICAL ]
        [ , PARALLEL = { SAFE | RESTRICTED | UNSAFE } ]
    )
    ```
    
    有序集聚集目前基本只能支持C语言编写,因此本文暂不介绍


## 最普通的聚集函数

我们从最普通的聚集函数开始,其他各个细分都是其上的扩展.

普通聚集函数的定义方式如下:

```sql
CREATE AGGREGATE name ( [ argmode ] [ argname ] arg_data_type [ , ... ] ) (
    SFUNC = sfunc,
    STYPE = state_data_type
    [ , SSPACE = state_data_size ]
    [ , FINALFUNC = ffunc ]
    [ , FINALFUNC_EXTRA ]
    [ , FINALFUNC_MODIFY = { READ_ONLY | SHAREABLE | READ_WRITE } ]
    [ , INITCOND = initial_condition ]
)
```

### 基本参数和执行流程

一个简单的聚集函数由一个或者多个普通函数组成:

+ 一个状态转移函数`SFUNC`,其形式为`sfunc( internal-state, next-data-values ) ---> next-internal-state`,
+ 一个可选的最终计算函数`FINALFUNC`,其形式为`ffunc( internal-state ) ---> aggregate-value`,如果指定了`FINALFUNC_EXTRA`则除了最终状态值和任何直接参数之外,最终函数还接收额外的对应于该聚集的常规(聚集)参数的`NULL`值.这主要用于在定义了一个多态聚集时允许正确地决定聚集的结果类型.

pg会在执行聚集函数时创建一个数据类型为`STYPE`的临时变量来保存聚集的当前内部状态.对每一个输入行,聚集参数值会被`SFUNC`计算,它用当前状态值和新参数值计算一个新的内部状态值.等所有行都被处理完后如果有设置`FINALFUNC`会调用一次`FINALFUNC`函数来计算该聚集的返回值;如果没有则最终的状态值会被返回.

就像reduce一样,我们可以通过设置`INITCOND`为聚集函数提供一个初始值,它被作为一个类型为`text`的值指定并且存储在数据库中(即`STYPE`指定类型值的字面量).但是它必须是状态值数据类型的一个常量的合法外部表示.如果没有提供,则状态值从NULL开始.我们也可以声明`SSPACE`来告诉pg这个函数的状态值的近似平均尺寸(以字节计).如果这个参数被忽略或者为零将使用一个基于设置项`state_data_type`的默认估计值.这个设置用于帮助规划器估计一个分组聚集查询所需的内存.只有估计哈希表能够放在`work_mem`大小的内存中时规划器才会对这类查询使用`哈希聚集`.因此对这个参数设置大的值会阻止使用哈希聚集.

需要注意指定的`SFUNC`和`FINALFUNC`被声明为`STRICT`的情况下执行流程会有一些变化

+ 如果`SFUNC`被声明为`strict`且如果初始状态值就是`NULL`,那么碰到第一个没有空值的行时状态值会被替换成第一个参数值,并且对于每一个后续的没有空值的行都会调用`SFUNC`.需要注意只有当`STYPE`和第一个`arg_data_type`相同时这种行为才可用.当这些类型不同时你必须提供一个非空初始条件或者使用一个非严格转移函数.

+ 如果`FINALFUNC`被声明为`strict`,那么当最终状态值为空时将不会调用它而是自动地返回一个`NULL`.在任何情况下最终函数都可以返回一个空值.

`FINALFUNC_MODIFY = { READ_ONLY | SHAREABLE | READ_WRITE }`用于指定`FINALFUNC`是否为不会修改参数的纯函数.`READ_ONLY`表示它不会修改;其他两个值表示它可能会更改迁移状态值.在普通聚集函数定义时默认取值为`READ_ONLY`
 

### 例子

一个典型的例子是`sum`,我们可以像下面这样为complex类型定义个简单的累积求和聚集函数

```sql
CREATE AGGREGATE sum (complex)
(
    sfunc = complex_add,
    stype = complex,
    initcond = '(0,0)'
);
```

它的调用方式也和正常的聚集函数一样.

## 普通聚集函数的移动聚集模式

聚集函数可以选择性地支持移动聚集模式,如果一个普通聚集函数支持移动聚集模式,那么它就可以被用在窗口查询上.

允许很大程度上提高在具有移动帧起点的窗口中执行的聚集函数的速度.基本思想是在通常的"前向执行"的`SFUNC`之外,聚集提供一个逆向转换函数`MINVFUNC`,该函数允许当行退出窗口帧时从聚集的运行状态值中移除它们的值.例如一个sum聚集使用加法作为前向转换函数,它可以使用减法作为逆向转换函数.如果没有一个逆向转换函数,每一次帧起点移动时窗口函数机制必须重新从头计算该聚集,这会导致运行时间与输入行的数量乘以平均帧长度成比例.如果有一个逆向转换函数,运行时间只与输入行的数量成比例.

当前状态值和包含在当前状态中最早的行的聚集输入值被传递给逆向转换函数.它必须重新构建出如果给定的输入行不再被聚集(只聚集其后的行)时状态值会是什么样.这有时要求前向转换函数保存比普通聚集模式下更多的状态.因此移动聚集模式使用一种完全不同于普通模式的实现--它有自己的状态数据类型,自己的前向转换函数以及自己的状态函数(如果需要).如果不需要额外的状态，这些可以和普通模式的数据类型和函数相同.

移动聚集函数自用的参数都以`M`开头,其完整定义方式如下:

```sql
CREATE AGGREGATE name ( [ argmode ] [ argname ] arg_data_type [ , ... ] ) (
    SFUNC = sfunc,
    STYPE = state_data_type
    [ , SSPACE = state_data_size ]
    [ , FINALFUNC = ffunc ]
    [ , FINALFUNC_EXTRA ]
    [ , FINALFUNC_MODIFY = { READ_ONLY | SHAREABLE | READ_WRITE } ]
    [ , INITCOND = initial_condition ]
    -- 下面是移动聚集函数的参数
    [ , MSFUNC = msfunc ]
    [ , MINVFUNC = minvfunc ]
    [ , MSTYPE = mstate_data_type ]
    [ , MSSPACE = mstate_data_size ]
    [ , MFINALFUNC = mffunc ]
    [ , MFINALFUNC_EXTRA ]
    [ , MINITCOND = minitial_condition ]
)
```

其中

+ `MSTYPE`对应普通模式下的`STYPE`,是移动聚集模式下状态值的类型;与之相应的`MINITCOND`对应普通模式下的`INITCOND`,用于保存在移动聚集模式下的状态值的初始字面量.
+ `MSFUNC`对应普通模式下的`SFUNC`,是"前向执行"的状态转移函数,其形式为`msfunc( internal-state, next-data-values ) ---> next-internal-state`,与之相应的`MSSPACE `对应普通模式下的`SSPACE`
+ `MFINALFUNC`对应普通模式下的`FINALFUNC`,其形式为`mffunc( internal-state ) ---> aggregate-value`.相应的`MFINALFUNC_EXTRA`对应普通模式下的`FINALFUNC_EXTRA`

+ `MINVFUNC`我们已经介绍过,逆向状态转移函数,其形式为`minvfunc( internal-state, next-data-values ) ---> next-internal-state`和`MSFUNC`一致


### 例子

我们修改上面的例子修改sum让他支持移动模式:


```sql
CREATE AGGREGATE sum (complex)
(
    sfunc = complex_add,
    stype = complex,
    initcond = '(0,0)',
    msfunc = complex_add,
    minvfunc = complex_sub,
    mstype = complex,
    minitcond = '(0,0)'
);
```

## 普通聚集函数部分聚集模式与并行聚集


聚集函数通常用在统计计算上,我们可以设置它让他支持并行计算从而提高查询的性能.这在loap场景下回非常有用.

一个聚集函数可以支持部分聚集.部分聚集的思想是在输入数据的不同子集上独立的运行该聚集的状态转移函数,然后把从这些子集得到的状态值组合起来产生最终的状态值,这样得到的状态值与在单次聚集操作中扫描所有输入得到的状态值相同.这种模式可以被用来进行并行聚集,用不同的工作者进程扫描表的不同部分.每一个工作者产生一个部分状态值,最后把这些部分状态值组合产生最终状态值.这种分治思想正是为并行而生的.

我们可以用以下方式定义一个支持部分聚集模式,可以并行化的普通聚集函数:

```sql
CREATE AGGREGATE name ( [ argmode ] [ argname ] arg_data_type [ , ... ] ) (
    SFUNC = sfunc,
    STYPE = state_data_type
    [ , SSPACE = state_data_size ]
    [ , FINALFUNC = ffunc ]
    [ , FINALFUNC_EXTRA ]
    [ , FINALFUNC_MODIFY = { READ_ONLY | SHAREABLE | READ_WRITE } ]
    [ , INITCOND = initial_condition ]
    -- 下面是移动聚集函数的参数
    [ , COMBINEFUNC = combinefunc ]
    [ , SERIALFUNC = serialfunc ]
    [ , DESERIALFUNC = deserialfunc ]
    PARALLEL = SAFE
)
```

### 设置并行计算

 
和普通自定义函数一样,自定义聚集函数也需要声明它自身是否并行安全,用的选项也一样是`PARALLEL = { SAFE | RESTRICTED | UNSAFE }`.

如果我们设置了`PARALLEL = SAFE`,那规划器就可以让聚集函数并行执行

### 聚集模式


为了支持部分聚集,聚集定义必须提供一个组合函数`COMBINEFUNC`(形式为`combinefunc(STYPE,STYPE)->STYPE`),这个函数接收两个该聚集的状态类型(表示在输入行的两个不同子集上得到的聚集结果)并且产生一个该状态类型的新值,该结果表示组合哪些聚集结果后的状态.至于来自两个集合的输入行的相对顺序则并没有指定.这意味着通常不可能为对输入行顺序敏感的聚集定义出可用的组合函数.

作为简单的例子,通过指定组合函数为与其转移函数中相同的"两者中较大者"和"两者中较小者"比较函数,`MAX`和`MIN`聚集可以支持部分聚集.`SUM`聚集则只需要一个额外的函数作为组合函数(同样,组合函数与其转移函数相同,除非状态值的宽度比输入数据类型更宽).

组合函数很像一个把状态类型值而不是底层输入类型值作为其第二个参数的转移函数.尤其是处理空值和严格函数的规则是相似的.此外如果聚集定义指定了非空的`initcond`,记住那不仅会被作为*每一次部分聚集运行的初始状态*,还会*被作为组合函数的初始状态*,对每一个部分结果都会调用组合函数将部分结果组合到该初始状态中.

如果聚集的状态类型被声明为`internal`,则组合函数应负责在用于聚集状态值的内存上下文中分配其结果.这意味着当第一个输入为`NULL`时不能简单地返回第二个输入,因为那个值将会在错误的上下文中并且将不具有足够的寿命.

当聚集的状态类型被声明为`internal`时,通常聚集定义提供序列化函数`SERIALFUNC(internal)-> bytea`和反序列化函数`DESERIALFUNC(bytea,internal)-> internal`也是合适的(参数中的`internal`是无用的,但是为了类型安全的原因还是要求有该参数),这两个函数允许这样一种状态值被从一个进程复制到另一个进程.如果没有这些函数就无法执行并行聚集,并且未来的本地/远程聚集之类的应用也可能无法工作.
